<a href="https://colab.research.google.com/github/mhdSharuk/Data-Science-Bowl-2K19/blob/master/DSB_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes

**Groupby data to get the number of attempts each installation_id played**


*   train_data.groupby(['game_session','installation_id'],as_index =False)['title'].agg({'value_counts'}).rename(columns={'value_counts':'Total_no'}).head()

*   test_data.groupby(['game_session','installation_id'])['title'].agg({'value_counts'}).rename(columns={'value_counts':'Total_no'}).index.get_level_values(3)



# Mounting Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Importing Required Modules

In [2]:
!pip install catboost

     |████████████████████████████████| 62.9MB 38kB/s 


In [0]:
import pandas as pd
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows', 28445)
import numpy as np
import os
import gc
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import json
import pprint
import seaborn as sns
from catboost import CatBoostClassifier,CatBoostRegressor,Pool,cv
import xgboost as xgb
import lightgbm as lgb

# Helper Functions

In [0]:
def read_file():
  specs = pd.read_csv('/content/drive/My Drive/specs.csv')
  train_labels = pd.read_csv('./drive/My Drive/train_labels.csv.zip')
  train = pd.read_csv('./drive/My Drive/train.csv.zip')
  test = pd.read_csv('./drive/My Drive/test.csv.zip')
  sub = pd.read_csv('./drive/My Drive/sample_submission.csv')
  
  return specs,train_labels,train,test,sub

In [0]:
specs,train_labels,train,test,sub = read_file()

In [0]:
def get_datetime(df):
  df['timestamp'] = pd.to_datetime(df['timestamp'])
  df['date'] = df['timestamp'].dt.date
  df['month'] = df['timestamp'].dt.month
  df['hour'] = df['timestamp'].dt.hour
  df['day_of_week'] = df['timestamp'].dt.dayofweek
  
for c in [train,test]:
  get_datetime(c)
  c.pop('timestamp')
  


# Exploratatry Data Analysis

In [35]:
train_labels.head()

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


In [37]:
train.head()

,event_id,game_session,event_data,installation_id,event_count,event_code,game_time,title,type,world,date,month,year,day_of_week,hour
0,27253bdc,45bb1e1b6b50c07b,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,2019-09-06,9,2019,4,17
1,27253bdc,17eeb7f223665f53,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK,2019-09-06,9,2019,4,17
2,77261ab5,0848ef14a8dc6892,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,2019-09-06,9,2019,4,17
3,b2dba42b,0848ef14a8dc6892,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK,2019-09-06,9,2019,4,17
4,1bb5fbdb,0848ef14a8dc6892,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK,2019-09-06,9,2019,4,17


In [38]:
test.head()

,event_id,game_session,event_data,installation_id,event_count,event_code,game_time,title,type,world,date,month,year,day_of_week,hour
0,27253bdc,0ea9ecc81a565215,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,2019-09-10,9,2019,1,16
1,27253bdc,c1ea43d8b8261d27,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK,2019-09-10,9,2019,1,16
2,27253bdc,7ed86c6b72e725e2,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK,2019-09-10,9,2019,1,16
3,27253bdc,7e516ace50e7fe67,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES,2019-09-10,9,2019,1,16
4,7d093bf9,a022c3f60ba547e7,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES,2019-09-10,9,2019,1,16
